In [1]:
import sys
sys.path.append('../')

import tquant as tq
from engine.globalconfig import GlobalConfig
from engine.loader.curveload import MarketDataLoader

from datetime import date
import tensorflow as tf
import pandas as pd

# Settings

In [2]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)

In [3]:
tq.Settings.evaluation_date

datetime.date(2023, 9, 29)

In [4]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

# Market

In [5]:
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

rates_6m = mdl.ir_eur_curve_6m['quote'].values
times_6m = mdl.ir_eur_curve_6m['daycount'].values/365

In [6]:
estr_curve = tq.RateCurve(times, rates)
eur6m_curve = tq.RateCurve(times_6m, rates_6m)

# Fixed Leg

In [7]:
fixed_schedule = [tq.TARGET().advance(evaluation_date,
                         t,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing) for t in [0,6,12,18,24]]
notionals = [1e6] * (len(fixed_schedule) - 1)
coupon_rates = [0.03] * (len(fixed_schedule) - 1)

fixed_leg = tq.FixedRateLeg(fixed_schedule,
                            notionals,
                            coupon_rates,
                            daycounter= daycounter,
                            compounding=tq.CompoundingType.Simple,
                            frequency= tq.Frequency.Annual)

In [8]:
fixed_leg_pricer = tq.FixedLegDiscounting(fixed_leg)

In [9]:
fixed_leg_pricer.price(estr_curve, evaluation_date, tq.FixedCouponDiscounting)

<tf.Tensor: shape=(), dtype=float64, numpy=58131.51747537639>

In [10]:
fixed_leg_pricer.price(estr_curve, evaluation_date, tq.FixedCouponDiscounting)

<tf.Tensor: shape=(), dtype=float64, numpy=58131.51747537639>

In [11]:
npv, tape = fixed_leg_pricer.price_aad(estr_curve, evaluation_date, tq.FixedCouponDiscounting)
sensitivities = tape.gradient(npv, [estr_curve.rates, eur6m_curve.rates])

In [12]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0]]).T
sensy.columns = ['time', 'sensitivity']
sensy[sensy['time'] > 0.4].head(20)

,time,sensitivity
10,0.419178,NaN
11,0.49589,"tf.Tensor(-6850.130249995172, shape=(), dtype=..."
12,0.583562,"tf.Tensor(-584.0195832544138, shape=(), dtype=..."
13,0.665753,NaN
14,0.747945,NaN
15,0.832877,NaN
16,0.917808,NaN
17,1.005479,"tf.Tensor(-12517.389366366535, shape=(), dtype..."
18,1.084932,"tf.Tensor(-2669.3096440769896, shape=(), dtype..."
19,1.169863,NaN


# Floating Leg

In [13]:
spreads = [0] * (len(fixed_schedule) - 1)
gearing = [1] * (len(fixed_schedule) - 1)
floating_schedule = fixed_schedule
index = tq.IborIndex('EUR6M',
             calendar,
             6,
             tq.TimeUnit.Months)

floating_leg = tq.FloatingRateLeg(floating_schedule,
                               notionals,
                               gearing,
                               spreads,
                               index,
                               daycounter)

In [14]:
floating_leg_pricer = tq.FloatingLegDiscounting(floating_leg)

In [15]:
floating_leg_pricer.price(estr_curve, eur6m_curve, evaluation_date, tq.FloatingCouponDiscounting)

<tf.Tensor: shape=(), dtype=float64, numpy=73422.18552775549>

In [16]:
npv, tape =floating_leg_pricer.price_aad(estr_curve, eur6m_curve, evaluation_date, tq.FloatingCouponDiscounting)
sensitivities = tape.gradient(npv, [estr_curve.rates, eur6m_curve.rates])

In [17]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0], sensitivities[1]]).T
sensy.columns = ['time', 'sensy_estr', 'sensy_eur6m']
sensy[sensy['time'] > 0.2].head(10)

,time,sensy_estr,sensy_eur6m
8,0.249315,NaN,NaN
9,0.334247,None,"tf.Tensor(9426.124691723555, shape=(), dtype=f..."
10,0.419178,None,"tf.Tensor(391.7631616996441, shape=(), dtype=f..."
11,0.49589,"tf.Tensor(-9497.84399362655, shape=(), dtype=f...",None
12,0.583562,"tf.Tensor(-809.7549518824293, shape=(), dtype=...","tf.Tensor(17107.208223456284, shape=(), dtype=..."
13,0.665753,None,"tf.Tensor(1588.2482716945087, shape=(), dtype=..."
14,0.747945,NaN,NaN
15,0.832877,None,"tf.Tensor(28417.42868711357, shape=(), dtype=f..."
16,0.917808,None,"tf.Tensor(5877.083791738813, shape=(), dtype=f..."
17,1.005479,"tf.Tensor(-16950.189095604055, shape=(), dtype...",None
